In [19]:
import numpy as np
import sys

## With DualSimplexSolver

In [1]:
from functions.DualSimplexSolver import DualSimplexSolver
if __name__ == "__main__":
    try:
        example_obj_type = 'max'
        example_c = [-2, -3,-1]
        example_A = [
            [1, 4,2],
            [3, 2,0],
        ]
        example_relations = ['>=', '>=']
        example_b = [8,6]

        solver = DualSimplexSolver(example_obj_type, example_c, example_A, example_relations, example_b)
        solver.solve()
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        import traceback
        traceback.print_exc()

--- Starting Dual Simplex Method ---

--- Iteration 0 ---
      BV        Z       x1       x2       x3       s1       s2      RHS
------------------------------------------------------------------------
       Z    1.000    2.000    3.000    1.000    0.000    0.000    0.000
      s1    0.000   -1.000   -4.000   -2.000    1.000    0.000   -8.000
      s2    0.000   -3.000   -2.000   -0.000    0.000    1.000   -6.000
------------------------------------------------------------------------

Step: Select Pivot Row (Leaving Variable)
   RHS values (b): [-8. -6.]
   Most negative RHS is -8.000 in Row 1 (Basic Var: s1).
   Leaving Variable: s1 (Row 1)

Step: Select Pivot Column (Entering Variable) using Ratio Test
   Pivot Row (Row 1) coefficients (excluding Z, RHS): [-1. -4. -2.  1.  0.]
   Objective Row coefficients (excluding Z, RHS): [2. 3. 1. 0. 0.]
   Calculating ratios = ObjCoeff / abs(PivotRowCoeff) for PivotRowCoeff < 0:
      Var x1 (Col 1): Coeff=-1.000, ObjCoeff=2.000, Ratio = 2.0

## Without DualSimplexSolver

In [1]:
from functions.get_user_input import get_user_input
from functions.solve_lp_via_dual import solve_lp_via_dual
from functions.solve_primal_directly import solve_primal_directly


use_example = True # Set to False to use user input

if use_example:
     print("--- Using Example Problem ---")
     example_obj_type = 'max'
     example_c = [-2, -3,-1]
     example_A = [
          [1, 4,2],
          [3, 2,0],
     ]
     example_relations = ['>=', '>=']
     example_b = [8,6]

     print(f"Objective: {example_obj_type} Z = {' + '.join([f'{co:.2f}*x{i+1}' for i, co in enumerate(example_c)])}")
     print("Constraints:")
     for i in range(len(example_A)):
          print(f"  {' + '.join([f'{aij:.2f}*x{j+1}' for j, aij in enumerate(example_A[i])])} {example_relations[i]} {example_b[i]:.2f}")
     print("------------------------\n")

     obj_type, c, A, relations, b = example_obj_type, example_c, example_A, example_relations, example_b
else:
     try:
          obj_type, c, A, relations, b = get_user_input()
          print("\n--- Problem Received ---")
          print(f"Objective: {obj_type} Z = {' + '.join([f'{co:.2f}*x{i+1}' for i, co in enumerate(c)])}")
          print("Constraints:")
          for i in range(len(A)):
               print(f"  {' + '.join([f'{aij:.2f}*x{j+1}' for j, aij in enumerate(A[i])])} {relations[i]} {b[i]:.2f}")
          print("------------------------\n")
     except Exception as e:
          print(f"\nAn error occurred during input: {e}")
          exit()

# Solve using the dual method
status, message, primal_sol, dual_sol, obj_val = solve_lp_via_dual(obj_type, c, A, relations, b)

print("\n" + "="*40)
print("          FINAL RESULT")
print("="*40)
print(f"Status: {status} ({message})")
if status == 0:
     print(f"Optimal Primal Objective Value: {obj_val:.6f}")
     print("Primal Variables (x*):")
     if primal_sol:
          for var, val in primal_sol.items():
               print(f"   {var}: {val:.6f}")
     print("Dual Variables (p*):")
     if dual_sol:
          # Map dual vars back to original constraints if possible (tricky due to splitting '=')
          # For simplicity, just print the p's as solved.
          for var, val in dual_sol.items():
               print(f"   {var}: {val:.6f}") # p_i corresponds to the i-th *standardized* constraint
print("="*40)

# Optional: Solve directly for comparison
print("\n--- Solving Primal Directly (for comparison) ---")
status_direct, msg_direct, psol_direct, _, obj_direct = solve_primal_directly(obj_type, c, A, relations, b)
if status_direct == 0:
     print(f"Direct Solve Optimal Value: {obj_direct:.6f}")
     print("Direct Solve Primal Variables:")
     for var, val in psol_direct.items():
          print(f"   {var}: {val:.6f}")
else:
     print(f"Direct Solve Failed: {msg_direct}")
print("="*40)

--- Using Example Problem ---
Objective: max Z = -2.00*x1 + -3.00*x2 + -1.00*x3
Constraints:
  1.00*x1 + 4.00*x2 + 2.00*x3 >= 8.00
  3.00*x1 + 2.00*x2 + 0.00*x3 >= 6.00
------------------------

--- Step 1: Formulate the Dual Problem ---
   Primal is Max. Converting to Min by negating objective coefficients.
   Adjusting primal constraints to >= form for dual formulation:
      Constraint 1: Keeping as >= ( [1. 4. 2.] >= 8.0 )
      Constraint 2: Keeping as >= ( [3. 2. 0.] >= 6.0 )

   Dual Problem Formulation:
      Objective: Maximize p * [8.00, 6.00]
      Subject to:
         1.00*p1 + 3.00*p2 <= 2.00
         4.00*p1 + 2.00*p2 <= 3.00
         2.00*p1 + 0.00*p2 <= 1.00
      p_i >= 0 for i=1..2

--- Step 2: Solve the Dual Problem using SciPy linprog ---

   Optimal Dual Solution Found:
      Dual Variables (p*):
         p1 = 0.500000
         p2 = 0.500000
      Optimal Dual Objective Value (Max p*b): 7.000000

--- Step 3: Check Strong Duality ---
   Strong duality implies the op